In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data

from transformations import hitnet_trafo

In [8]:
from callback import Save

In [2]:
data1 = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols')

data2 = Data(dir='/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols')

In [3]:
hits_input = tf.keras.Input(shape=(5,))
params_input = tf.keras.Input(shape=(len(data1.labels),))
t = hitnet_trafo(labels=data1.labels)

h = t(hits_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(1024, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

hit_net = tf.keras.Model(inputs=[hits_input, params_input], outputs=outputs)
hit_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
hitnet_trafo (hitnet_trafo)     (None, 15)           0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           512         hitnet_trafo[0][0]               
______________________________________________________________________________________________

In [4]:
optimizer = tf.keras.optimizers.Adam(0.00001)
hit_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [5]:
train_id = 'HitNet_1024_L3' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [9]:
callbacks = []
callbacks.append(Save(save_every=10, path_template='models/'+train_id+'/epoch_%i'))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [10]:
train_data1, test_data1 = data1.get_hitnet_data(train_batch_size=4096,
                                                 test_batch_size=4096,
                                                )
train_data2, test_data2 = data2.get_hitnet_data(train_batch_size=4096,
                                                 test_batch_size=4096,
                                                )

In [11]:
train_data = tf.data.Dataset.concatenate(train_data1,train_data2)
test_data = tf.data.Dataset.concatenate(test_data1,test_data2)

In [12]:
hist = hit_net.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                  initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000


KeyboardInterrupt: 

In [ ]:
# L7 0.28
# L3 0.24